# Only for Colab

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/My Drive/Colab Notebooks/2019NN/

/content/gdrive/My Drive/Colab Notebooks/2019NN


In [1]:
!ls

data  data_helpers  Train_Hotel.ipynb  Train_Hotel.py


In [6]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


# Start

In [1]:
from fastai.text import *
from data_helpers.Data import *
from fastai.text.transform import *

In [2]:
sent_num_file = ["train.count", "test.count"]
rating_file = ["train.rating", "test.rating"]
content_file = ["train.txt", "test.txt"]

dataset_dir = "./data/beer_100k/"

In [3]:
def concat_to_doc(sent_list, sent_count):
    start_index = 0
    docs = []
    for s in sent_count:
        doc = " xxPERIOD ".join(sent_list[start_index:start_index + s])
        doc = doc + " xxPERIOD "
        docs.append(doc)
        start_index = start_index + s
    return docs

In [4]:
TRAIN_DATA = 0
TEST_DATA = 1

## Load Training Data

In [61]:
# Load Count
sent_count_train = list(open(dataset_dir + sent_num_file[TRAIN_DATA], "r").readlines())
sent_count_train = [int(s) for s in sent_count_train if (len(s) > 0 and s != "\n")]
print( sent_count_train[0:5] )

# Load Ratings
aspect_rating_train = list(open(dataset_dir + rating_file[TRAIN_DATA], "r").readlines())
aspect_rating_train = [s for s in aspect_rating_train if (len(s) > 0 and s != "\n")]

aspect_rating_train = [s.split(" ") for s in aspect_rating_train]
aspect_rating_train = np.array(aspect_rating_train)[:, :]
aspect_rating_train = aspect_rating_train.astype(np.float) - 1
# aspect_rating_train = np.rint(aspect_rating_train).astype(int)  # ROUND TO INTEGER =================
aspect_rating_train = pd.DataFrame(aspect_rating_train)
print( aspect_rating_train.head() )

# Load Sents
sents_train = list(open(dataset_dir + content_file[TRAIN_DATA], "r").readlines())
sents_train = [s.strip() for s in sents_train]
sents_train = [s[:-1] for s in sents_train if s.endswith(".")]
print( sents_train[0:5] )

[8, 7, 9, 14, 12]
     0    1    2    3    4
0  0.5  1.5  0.5  0.5  1.0
1  2.0  2.0  2.0  2.0  1.5
2  2.0  2.0  2.0  2.0  1.5
3  2.0  2.5  2.0  1.5  2.0
4  2.5  2.5  3.0  3.0  2.5
['A lot of foam', 'But a lot', 'In the smell some banana, and then lactic and tart', 'Not a good start', 'Quite dark orange in color, with a lively carbonation (now visible, under the foam)']


In [62]:
docs_train = concat_to_doc(sents_train, sent_count_train)

docs_train = pd.DataFrame(docs_train)

docs_train.head()

,0
0,A lot of foam xxPERIOD But a lot xxPERIOD In t...
1,"Dark red color, light beige foam, average xxPE..."
2,"Almost totally black xxPERIOD Beige foam, quit..."
3,"Golden yellow color xxPERIOD White, compact fo..."
4,"22 oz bottle from ""Lifesource"" Salem xxPERIOD ..."


In [63]:
docs_train.iloc[0][0]

'A lot of foam xxPERIOD But a lot xxPERIOD In the smell some banana, and then lactic and tart xxPERIOD Not a good start xxPERIOD Quite dark orange in color, with a lively carbonation (now visible, under the foam) xxPERIOD Again tending to lactic sourness xxPERIOD Same for the taste xxPERIOD With some yeast and banana xxPERIOD '

In [64]:
df_train = pd.concat( [aspect_rating_train, docs_train], axis=1, ignore_index=True )
df_train.head()

,0,1,2,3,4,5
0,0.5,1.5,0.5,0.5,1.0,A lot of foam xxPERIOD But a lot xxPERIOD In t...
1,2.0,2.0,2.0,2.0,1.5,"Dark red color, light beige foam, average xxPE..."
2,2.0,2.0,2.0,2.0,1.5,"Almost totally black xxPERIOD Beige foam, quit..."
3,2.0,2.5,2.0,1.5,2.0,"Golden yellow color xxPERIOD White, compact fo..."
4,2.5,2.5,3.0,3.0,2.5,"22 oz bottle from ""Lifesource"" Salem xxPERIOD ..."


In [65]:
df_train.iloc[:,0:5].min(axis=0)

0   -1.0
1   -1.0
2    0.0
3    0.0
4    0.0
dtype: float64

In [66]:
df_train.shape

(75116, 6)

In [67]:
df_train = df_train[df_train.iloc[:,0:5].min(axis=1) >= 0]

In [68]:
df_train.shape

(75113, 6)

## Load Testing Data

In [69]:
# Load Count
sent_count_test = list(open(dataset_dir + sent_num_file[TEST_DATA], "r").readlines())
sent_count_test = [int(s) for s in sent_count_test if (len(s) > 0 and s != "\n")]
print( sent_count_test[0:5] )

# Load Ratings
aspect_rating_test = list(open(dataset_dir + rating_file[TEST_DATA], "r").readlines())
aspect_rating_test = [s for s in aspect_rating_test if (len(s) > 0 and s != "\n")]

aspect_rating_test = [s.split(" ") for s in aspect_rating_test]
aspect_rating_test = np.array(aspect_rating_test)[:, :]
aspect_rating_test = aspect_rating_test.astype(np.float) - 1
# aspect_rating_test = np.rint(aspect_rating_test).astype(int)  # ROUND TO INTEGER =================
aspect_rating_test = pd.DataFrame(aspect_rating_test)
print( aspect_rating_test.head() )

# Load Sents
sents_test = list(open(dataset_dir + content_file[TEST_DATA], "r").readlines())
sents_test = [s.strip() for s in sents_test]

# Sents to Doc
docs_test = concat_to_doc(sents_test, sent_count_test)

docs_test = pd.DataFrame(docs_test)
print( docs_test.head() )

[13, 12, 7, 9, 6]
     0    1    2    3    4
0  3.0  3.0  3.5  3.0  3.5
1  2.0  2.5  2.5  2.0  2.5
2  3.0  3.0  3.0  2.5  3.0
3  3.0  2.0  2.0  1.5  3.5
4  2.0  1.5  2.0  2.0  2.0
                                                   0
0  According to the website, the style for the Ca...
1  Poured from the bottle into a Chimay goblet. x...
2  Notes from 6/24. xxPERIOD A: Bright golden glo...
3  22 oz. xxPERIOD bomber,. xxPERIOD A: Pours a c...
4  Brown in color, somewhere between a porter and...


In [70]:
docs_test.iloc[0][0]

"According to the website, the style for the Caldera Cauldron changes every year. xxPERIOD The current release is a DIPA, which frankly is the only cauldron I'm familiar with (it was an IPA/DIPA the last time I ordered a cauldron at the horsebrass several years back). xxPERIOD In any event. xxPERIOD at the Horse Brass yesterday. xxPERIOD The beer pours an orange copper color with good head retention and lacing. xxPERIOD The nose is all hoppy IPA goodness, showcasing a huge aroma of dry citrus, pine and sandlewood. xxPERIOD The flavor profile replicates the nose pretty closely in this West Coast all the way DIPA. xxPERIOD This DIPA is not for the faint of heart and is a bit much even for a hophead like myslf. xxPERIOD The finish is quite dry and hoppy, and there's barely enough sweet malt to balance and hold up the avalanche of hoppy bitterness in this beer. xxPERIOD Mouthfeel is actually fairly light, with a long, persistentely bitter finish. xxPERIOD Drinkability is good, with the alc

In [71]:
df_test = pd.concat( [aspect_rating_test, docs_test], axis=1, ignore_index=True )
df_test.head()

,0,1,2,3,4,5
0,3.0,3.0,3.5,3.0,3.5,"According to the website, the style for the Ca..."
1,2.0,2.5,2.5,2.0,2.5,Poured from the bottle into a Chimay goblet. x...
2,3.0,3.0,3.0,2.5,3.0,Notes from 6/24. xxPERIOD A: Bright golden glo...
3,3.0,2.0,2.0,1.5,3.5,"22 oz. xxPERIOD bomber,. xxPERIOD A: Pours a c..."
4,2.0,1.5,2.0,2.0,2.0,"Brown in color, somewhere between a porter and..."


In [53]:
df_test.shape

(24884, 6)

In [72]:
df_test.iloc[:,0:5].min(axis=0)

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
dtype: float64

# Build Language Model Data Bunch

In [14]:
lmdb = TextLMDataBunch.from_df("./data/", train_df=df_train, valid_df=df_test, text_cols=5, label_cols=[0,1,2,3,4])

In [15]:
len( lmdb.vocab.itos )

31600

In [16]:
lmdb.vocab.itos[0:8]

['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxmaj', 'xxup', 'xxrep']

In [17]:
lmdb.show_batch()

idx,text
0,"xxbos xxmaj dark red color , light beige foam , average xxperiod xxmaj in the smell malt and caramel , not really light xxperiod xxmaj again malt and caramel in the taste , not bad in the end xxperiod xxmaj maybe a note of honey in teh back , and a light fruitiness xxperiod xxmaj average body xxperiod xxmaj in the aftertaste a light bitterness , with the malt and"
1,"balanced , but yet again , i could n't detect the rosemary in the beer xxperiod xxmaj none xxperiod xxmaj zip xxperiod xxmaj zilch xxperiod xxmaj nada xxperiod xxmaj the mouthfeel was fine xxperiod xxmaj it 's well carbonated and tastes fine right down to the last drop xxperiod xxmaj overall , i just wish they 'd added more rosemary to the beer xxperiod xxmaj rosemary is such an assertive"
2,"xxperiod xxmaj the beer is extremely smooth xxperiod xxmaj overall this beer is excellent xxperiod i 'm not sure i have had a barrel - aged beer , barleywine or otherwise , that has been this well rounded and complex so close to bottling xxperiod xxmaj this one did n't come with all the hype and fanfare of some beers of this stature , in fact i only picked it"
3,"meet up tasting in xxmaj bakersfield at xxmaj imbibe xxmaj wine xxmaj bar xxperiod xxmaj it poured nicely , smallish head , pitch black and a tinge of chocolate brown for the head xxperiod xxmaj the smell , whilst alcoholic , was not overpowering xxperiod i smelled vanilla , dark chocolate , caramel , flavored coffee too xxperiod xxmaj now comes the fun part xxperiod the taste xxperiod xxmaj it"
4,"4 s : 3 xxperiod 5 t : 4 m : 4 o : 3 xxperiod 5 xxperiod xxmaj update : 09 / 2011 xxperiod xxmaj bottle purchased in xxmaj cambridge , xxup uk in xxmaj may 2011 xxperiod opened in xxmaj september xxperiod xxmaj just as delicious , pleasant , and delightful as i remember it being in ' 07 xxperiod i 've adjusted my ratings numbers to reflect"


In [18]:
moms = (0.8,0.7)

In [19]:
learn = language_model_learner(lmdb, AWD_LSTM)
learn.unfreeze()
learn.fit_one_cycle(10, slice(1e-2), moms=moms)

FREEZING
Sequential(
  (0): WeightDropout(
    (module): LSTM(400, 1152, batch_first=True)
  )
  (1): RNNDropout()
)
FREEZING
Sequential(
  (0): WeightDropout(
    (module): LSTM(1152, 1152, batch_first=True)
  )
  (1): RNNDropout()
)
FREEZING
Sequential(
  (0): WeightDropout(
    (module): LSTM(1152, 400, batch_first=True)
  )
  (1): RNNDropout()
)
UN-FREEZING
Sequential(
  (0): Embedding(31600, 400, padding_idx=1)
  (1): EmbeddingDropout(
    (emb): Embedding(31600, 400, padding_idx=1)
  )
  (2): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=31600, bias=True)
    (output_dp): RNNDropout()
  )
)
UN-FREEZING
Sequential(
  (0): WeightDropout(
    (module): LSTM(400, 1152, batch_first=True)
  )
  (1): RNNDropout()
)
UN-FREEZING
Sequential(
  (0): WeightDropout(
    (module): LSTM(1152, 1152, batch_first=True)
  )
  (1): RNNDropout()
)
UN-FREEZING
Sequential(
  (0): WeightDropout(
    (module): LSTM(1152, 400, batch_first=True)
  )
  (1): RNNDropout()
)
UN-FREEZING
Se

epoch,train_loss,valid_loss,accuracy,time
0,3.418838,3.557895,0.337829,05:42
1,3.252795,3.440653,0.352359,05:51
2,3.171182,3.391150,0.361088,05:51
3,3.095272,3.347114,0.359018,06:01
4,3.038630,3.333527,0.364209,05:51
5,3.002490,3.295170,0.366441,05:51
6,2.959563,3.276685,0.370258,05:51
7,2.890160,3.261306,0.372872,05:50
8,2.876720,3.253816,0.372780,05:51
9,2.862697,3.254771,0.372463,05:52


In [20]:
lmdb.save('lm_databunch_beer.1115')

In [21]:
learn.save('lm_learner_beer.1115')

In [22]:
learn.save_encoder('lm_enc_beer.1115')

# Build Classifier Data Bunch

In [73]:
np.random.seed(42)
msk = np.random.rand(len(df_train)) < 0.85
df_train08 = df_train[msk]
df_valid02 = df_train[~msk]
len(df_train08), len(df_valid02)

(63909, 11204)

In [74]:
clas_db_tvt = TextClasDataBunch.from_df("./data/",
                                     train_df=df_train08,
                                     valid_df=df_valid02,
                                     test_df=df_test,
                                     text_cols=5,
                                     label_cols=[0,1,2,3,4],
                                     vocab=lmdb.train_ds.vocab,
                                     bs=42)

In [45]:
# clas_db_tv = TextClasDataBunch.from_df("./data/",
#                                      train_df=df_train,
#                                      valid_df=df_test,
#                                      text_cols=5,
#                                      label_cols=[0,1,2,3,4],
#                                      vocab=lmdb.train_ds.vocab,
#                                      bs=42)

In [75]:
clas_db_tvt.show_batch()

text,target
xxbos xxmaj watching some football tonight ( xxup xxunk ) and having the last of my 2005 and 2006 xxmaj bigfoots ( xxmaj bigfeet ? ? ) xxperiod xxmaj lots of malt in there xxperiod toffee and caramel xxperiod very xxmaj english - like these days xxperiod quite tasty xxperiod xxmaj just one 2004 left xxperiod :-( xxmaj cheers ! ! xxperiod * * * * * * xxperiod xxmaj,
"xxbos xxmaj acquired this beauty thanks to beerchitect xxperiod i 've been dreaming of cracking this open for weeks xxperiod i 've woken up in a cold sweat a few times dreaming of dancing xxmaj founders bottles with xxup kbs letters flying out at me and into my mouth , and rivers of dark tasty liquid showering me in droves of awesomeness xxperiod xxmaj and now its finally time to",
"xxbos 750 ml bottle from xxmaj wine xxmaj warehouse , poured in to a cervoise xxperiod xxmaj appearance : xxmaj pours a thoroughly hazed tangerine peach , with ( unusually ) one large stream of fast , densely - packed carbonation moving not nearly as fast as i would expect from a beer with this much head xxperiod xxmaj that is , the beer looks it 's gon na be",
"xxbos xxmaj picked up this 750 from xxmaj hop xxmaj city out in xxmaj atlanta xxperiod xxmaj the description on the bottle caught my eye , so i figured what the hell xxperiod i think the bottle was about $ 9 xxperiod 00 or so xxperiod xxmaj served up in a tulip xxperiod a : xxmaj pours a very intersting light yellow in color , almost champagne like in appearance",
"xxbos 2008 bottling xxperiod i 've been sitting on this since xxmaj january , when i decided it was a bit too rough for me at the time xxperiod xxmaj hopefully four months has mellowed it more to my tastes xxperiod xxmaj looks as good as i remember from before when poured into a goblet xxperiod xxmaj like a cherry - wood color , with strawberry red brightness when held",


In [76]:
clas_db_tvt.save("beer_clas_databunch_float.TraValTes")

In [46]:
# clas_db_tv.save("beer_clas_databunch_rint.TraVal")

##  -- load LM Databunch and LM Learner

In [4]:
lmdb = load_data("./data/", "hotel_lm_databunch.1001")

In [5]:
learn = language_model_learner(lmdb, AWD_LSTM)
learn.unfreeze()
learn = learn.load("lang_model_hotel")

freezing:WeightDropout(
  (module): LSTM(400, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 400, batch_first=True)
)
freezing:RNNDropout()


## -- load databunch

In [6]:
clas_db = load_data("./data/", "hotel_cls_databunch.aspect_only")

In [7]:
clas_db.batch_size=2

In [8]:
clas_db.c

5

In [14]:
cls_learn = text_classifier_learner(clas_db, AWD_LSTM)

freezing:Embedding(23008, 400, padding_idx=1)
freezing:EmbeddingDropout(
  (emb): Embedding(23008, 400, padding_idx=1)
)
freezing:WeightDropout(
  (module): LSTM(400, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 400, batch_first=True)
)
freezing:RNNDropout()


# Model Test

In [8]:
encoder = learn.model[0]
encoder.reset()

In [31]:
learn.predict("the finish is ", n_words=20)

"the finish is  kinda sweet xxperiod very nice , but not overly complex xxperiod it 's a dark muddled brown with good head"